# Chapter 5 - Basic Structured Operations

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("chapter5").getOrCreate()

## Schemas

In [4]:
df = spark.read.format("json").load("../data/flight-data/json/2015-summary.json")
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [6]:
print(df.schema)

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))


A schema can also be manually specified

In [10]:
from pyspark.sql.types import StructType, StructField, StringType, LongType

myManualSchema = StructType([StructField("DEST_COUNTRY_NAME", StringType(), True), 
                             StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
                             StructField("count", LongType(), False, metadata={"hello":"world"})])

df = spark.read.format("json").schema(myManualSchema).load("../data/flight-data/json/2015-summary.json")
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)

